# December 16, 2023

https://adventofcode.com/2023/day/16

I replaced all \ in input with L to simplify escaping backslashes

In [338]:
test = f'''.|...L....
|.-.L.....
.....|-...
........|.
..........
.........L
..../.LL..
.-.-/..|..
.|....-|.L
..//.|....'''

test = test.split("\n")
test = [ [y for y in line] for line in test]

In [339]:
fn = "../data/2023/16.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz = [x.strip() for x in text]
puzz = [ [y for y in line] for line in puzz ]

In [340]:
test

[['.', '|', '.', '.', '.', 'L', '.', '.', '.', '.'],
 ['|', '.', '-', '.', 'L', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '|', '-', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '|', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'L'],
 ['.', '.', '.', '.', '/', '.', 'L', 'L', '.', '.'],
 ['.', '-', '.', '-', '/', '.', '.', '|', '.', '.'],
 ['.', '|', '.', '.', '.', '.', '-', '|', '.', 'L'],
 ['.', '.', '/', '/', '.', '|', '.', '.', '.', '.']]

### Part 1

In [378]:
class Beam:
    num_generated = 0

    def __init__( self, x=0, y=0, dir="r" ):
        self.x = x
        self.y = y
        self.dir = dir
        self.id = Beam.num_generated
        Beam.num_generated += 1

    def __str__( self ):
        if self.dir == "r":
            ch = ">"
        elif self.dir == "l":
            ch = "<"
        elif self.dir == "u":
            ch = "^"
        else:
            ch = "v"

        return f'''Beam {self.id} {ch} @({self.x},{self.y})'''

class MirrorMaze:
    def __init__( self, arr, x0=-1, y0=0, d0="r" ):
        self.map = arr
        self.ncol = len(arr[0])
        self.nrow = len(arr)

        self.beams = list()
        self.history = {}
        self.energy = 0

        # if d0 == "r":
        # # edge cases
        #     if self.map[y0][x0] in ["L", "|"]:
        #         dir = "d"
        #     elif self.map[y0][x0] == "/":
        #         dir = "u"
        #     else:
        #         dir = "r"
        # elif d0 == "l":
        #     if self.map[y0][x0] in ["L", "|"]:
        #         dir = "u"
        #     elif self.map[y0][x0] == "/":
        #         dir = "u"
        #     else:
        #         dir = "r"

        
        self.newBeam( x=x0, y=y0, dir=d0, outside=True )

        

    def tileId( self, x, y):
        return x*self.ncol + y
    
    def addHistory( self, x, y, dir ):
        tile = self.tileId(x,y)
        if tile in self.history.keys():
            if dir not in self.history[tile]:
                self.history[tile] += dir
            # else don't add again
        else:
            self.energy += 1
            self.history[tile] = dir

    def newBeam( self, x, y, dir, outside = False ):
        tile = self.tileId(x,y)

        if outside:
            # this is an "edge" case for the initial condition
            # we allow the beam to be outside the grid.
            self.beams.append( Beam(x,y,dir) )
        else:
            if (
                x < 0 or x >= self.ncol or
                y < 0 or y >= self.nrow or
                (tile in self.history.keys() and dir in self.history[tile])
            ):
                return
                      
            self.beams.append( Beam(x,y,dir) )
            self.addHistory(x,y,dir)

    def run( self ):
        while len(self.beams) > 0:
            self.stepBeam( 0 )
        return
        
    def endBeam( self, id = 0 ):
        #self.beams[id] = None
        self.beams = self.beams[:id] + self.beams[(id+1):]

    def stepBeam( self, id = 0 ):
        if id > len(self.beams):
            return
        
        b = self.beams[id]
        if b.dir == "r":
            b.x += 1
        elif b.dir == "l":
            b.x -= 1
        elif b.dir == "u":
            b.y -= 1
        else:
            b.y += 1

        # beam went off the map, remove it from tracking
        if b.x < 0 or b.x >= self.ncol or b.y < 0 or b.y >= self.nrow:
            self.endBeam(id)
            return
        
        tile = self.tileId( b.x, b.y )
        if tile in self.history.keys():
            # we're already tracing (or finished tracing) this path
            if b.dir in self.history[tile]:
                self.endBeam(id)
                return
            
        # add this dir to list of histories we've traced for this tile
        self.addHistory( b.x, b.y, b.dir )

        sym = self.map[b.y][b.x]

        if sym == "/":
            if b.dir == "r":
                b.dir = "u"
            elif b.dir == "l":
                b.dir = "d" 
            elif b.dir == "u":
                b.dir = "r" 
            else:
                b.dir = "l"
        elif sym == "L":
            if b.dir == "r":
                b.dir = "d"
            elif b.dir == "l":
                b.dir = "u" 
            elif b.dir == "u":
                b.dir = "l" 
            else:
                b.dir = "r"
        elif sym == "-" and (b.dir == "u" or b.dir == "d"):
            # add left/right beams if needed
            self.newBeam( b.x, b.y, "l" )
            self.newBeam( b.x, b.y, "r" )
            # remove the old beam
            self.endBeam(id)
            
        elif sym == "|" and (b.dir == "r" or b.dir == "l"):
            # add up/down beams
            self.newBeam( b.x, b.y, "d" )
            self.newBeam( b.x, b.y, "u" )
            # remove the old beam
            self.endBeam(id)

    def print( self, show="map" ):
        if show == "map":
            print( "\n".join( [ "".join(line) for line in self.map] ) )

        if show == "laser":
            for y in range(self.nrow):
                line = ""
                for x in range(self.ncol):
                    sym = self.map[y][x]
                    if sym == ".":
                        tile = self.tileId( x, y )
                        if tile not in self.history.keys():
                            line += "."
                        elif len( self.history[tile] ) > 1:
                            line += str( len(self.history[tile]) )
                        elif self.history[tile] == "r":
                            line += ">"
                        elif self.history[tile] == "l":
                            line += "<"
                        elif self.history[tile] == "u":
                            line += "^"
                        else:
                            line += "v"
                    else:
                        line += sym
                print(line)
        
        if show == "energy":
            for y in range(self.nrow):
                line = ""
                for x in range(self.ncol):
                    tile = self.tileId(x,y)
                    line += "#" if tile in self.history.keys() else "."
                print(line)

        print("\n")

In [379]:

t = MirrorMaze(test)

In [380]:
print( t.beams[0] )

Beam 0 > @(-1,0)


In [381]:
t.stepBeam()
t.print("laser")

>|...L....
|.-.L.....
.....|-...
........|.
..........
.........L
..../.LL..
.-.-/..|..
.|....-|.L
..//.|....




In [382]:
t.run()
print(t.energy)
t.print("laser")
t.print("energy")

46
>|<<<L....
|v-.L^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..L
.v../2LL..
<->-/vv|..
.|<<<2-|.L
.v//.|.v..


######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..




In [ ]:
p = MirrorMaze(puzz)
p.run()
print(p.energy)
p.print("laser")
p.print("energy")

### Part 2

In [386]:
def part2( puzz ):
    ncol = len(puzz[0])
    nrow = len(puzz)
    best = 0
    best_result = None

    for y in range( nrow ):
        p = MirrorMaze( puzz, x0=-1, y0=y, d0="r" )
        p.run()
        if p.energy > best:
            best = p.energy
            best_result = p
        
        p = MirrorMaze( puzz, x0=ncol, y0=y, d0="l" )
        p.run()
        if p.energy > best:
            best = p.energy
            best_result = p

    for x in range( ncol ):
        p = MirrorMaze( puzz, x0=x, y0=-1, d0="d" )
        p.run()
        if p.energy > best:
            best = p.energy
            best_result = p
        
        p = MirrorMaze( puzz, x0=x, y0=nrow, d0="u" )
        p.run()
        if p.energy > best:
            best = p.energy
            best_result = p

    return best, best_result

In [387]:
b, br = part2( test )
print(b)
br.print("laser")
br.print("energy")

51
.|<2<L....
|v-vL^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..L
.v.v/2LL..
<-2-/vv|..
.|<<<2-|.L
.v//.|.v..


.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..




In [388]:
ans, _ = part2( puzz )
print(ans)

6766
